In [2]:
import sympy as sym

In [3]:
n = 5

#M = sym.MatrixSymbol('M', n, n)
Q = sym.MatrixSymbol('Q', n, n)
L = sym.MatrixSymbol(r'\Lambda', n, n)
z = sym.MatrixSymbol('z', n, 1)

M = Q @ L @ Q.T
f = z.T @ M @ z
f.diff(L)

Q.T*z*z.T*Q

In [29]:
x, y, z, a = sym.symbols('x y z a')
func = sym.sqrt(a**2 + (x-y)**2) 
H = sym.hessian(func, (x, y))
H.simplify()
H

Matrix([
[ a**2/(a**2 + (x - y)**2)**(3/2), -a**2/(a**2 + (x - y)**2)**(3/2)],
[-a**2/(a**2 + (x - y)**2)**(3/2),  a**2/(a**2 + (x - y)**2)**(3/2)]])

In [31]:
func.diff(y)

(-x + y)/sqrt(a**2 + (x - y)**2)

In [26]:
H.det()

0

In [27]:
H[0,0]

a**2/(a**2 + (x - y)**2)**(3/2)

In [96]:
import numpy as np
import scipy as sp

def BFGS(
    grad,
    loss,
    x_0,
    max_iter=1000,
    tol=0.
):  
    N = len(x_0)
    H_k = np.eye(N)
    x_k = x_0.copy()
    g_k = grad(x_k)
    
    for k in range(max_iter):
        h_k = - H_k @ g_k
        
        alpha_k = sp.optimize.line_search(loss, grad, x_k, h_k)[0]
        
        x_k_1 = x_k + alpha_k * h_k
        
        s_k = x_k_1 - x_k
        
        g_k_1 = grad(x_k_1)
        
        y_k = g_k_1 - g_k
        
        rho_k = 1./(y_k @ s_k)
        H_k_1 = H_k - rho_k * (
            np.einsum('i,j,jk->ik', s_k, y_k, H_k, optimize='optimal') + 
            np.einsum('ij,j,k->ik', H_k, y_k, s_k, optimize='optimal') - 
            rho_k * np.einsum('i,j,jk,k,l->il', s_k, y_k, H_k, y_k, s_k, optimize='optimal') -
            np.outer(s_k, s_k))
        
        x_k = x_k_1
        g_k = g_k_1
        H_k = H_k_1
        
        if np.linalg.norm(g_k) < tol:
            break
        
    return x_k


def bfgs_method(f, fprime, x0, maxiter=None, epsi=10e-3):
    """
    Minimize a function func using the BFGS algorithm.
    
    Parameters
    ----------
    func : f(x)
        Function to minimise.
    x0 : ndarray
        Initial guess.
    fprime : fprime(x)
        The gradient of `func`.
    """
    
    if maxiter is None:
        maxiter = len(x0) * 200

    # initial values
    k = 0
    gfk = fprime(x0)
    N = len(x0)
    # Set the Identity matrix I.
    I = np.eye(N, dtype=int)
    Hk = I
    xk = x0
   
    while np.linalg.norm(gfk) > epsi and k < maxiter:
        
        # pk - direction of search
        
        pk = -np.dot(Hk, gfk)
        
        # Line search constants for the Wolfe conditions.
        # Repeating the line search
        
        # line_search returns not only alpha
        # but only this value is interesting for us

        line_search = sp.optimize.line_search(f, fprime, xk, pk)
        alpha_k = line_search[0]
        
        xkp1 = xk + alpha_k * pk
        sk = xkp1 - xk
        xk = xkp1
        
        gfkp1 = fprime(xkp1)
        yk = gfkp1 - gfk
        gfk = gfkp1
        
        k += 1
        
        ro = 1.0 / (np.dot(yk, sk))
        #print(ro)
        A1 = I - ro * sk[:, np.newaxis] * yk[np.newaxis, :]
        A2 = I - ro * yk[:, np.newaxis] * sk[np.newaxis, :]
        Hk = np.dot(A1, np.dot(Hk, A2)) + (ro * sk[:, np.newaxis] *
                                                 sk[np.newaxis, :])
        
    return (xk, k)
        

In [97]:
def f(x):
    return x[0]**2 - x[0]*x[1] + x[1]**2 + 9*x[0] - 6*x[1] + 20


# Derivative
def f1(x):
    return np.array([2 * x[0] - x[1] + 9, -x[0] + 2*x[1] - 6])

In [101]:
%%timeit
bfgs_method(f, f1, np.array([1., 1.]), epsi=1e-8)

163 μs ± 3.74 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [100]:
%%timeit
BFGS(
    grad=f1,
    loss=f,
    x_0=np.array([1., 1.]),
    tol=1e-8
)

2.86 ms ± 148 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [45]:
x = np.random.randn(5)
y = np.random.randn(5)
A = np.random.randn(5, 5)
np.einsum('i,ij,j->', x, A, y), x.T @ A @ y

(-11.578446687712411, -11.578446687712411)